In [25]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Binarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# === 1. Load Datasets ===
air = pd.read_csv('airquality.csv')
heart = pd.read_csv('heart.csv')  # Assuming heart disease dataset exists

# === 2. Data Cleaning ===
# Drop unnamed/index columns
air.drop(['Unnamed: 0'], axis=1, inplace=True, errors='ignore')
heart.drop(['Unnamed: 0'], axis=1, inplace=True, errors='ignore')

# Check and fill nulls
air.fillna(method='pad', inplace=True)
heart.fillna(method='pad', inplace=True)

# Drop remaining nulls (if any)
air.dropna(inplace=True)
heart.dropna(inplace=True)

# === 3. Output After Cleaning ===
print("Cleaned Dataset Overview\n")

print("Air Quality Dataset:")
print("Shape:", air.shape)
print("Missing values:\n", air.isnull().sum())
print("\nFirst 5 rows:\n", air.head())

print("\nHeart Disease Dataset:")
print("Shape:", heart.shape)
print("Missing values:\n", heart.isnull().sum())
print("\nFirst 5 rows:\n", heart.head())

Cleaned Dataset Overview

Air Quality Dataset:
Shape: (153, 6)
Missing values:
 Ozone      0
Solar.R    0
Wind       0
Temp       0
Month      0
Day        0
dtype: int64

First 5 rows:
    Ozone  Solar.R  Wind  Temp  Month  Day
0   41.0    190.0   7.4    67      5    1
1   36.0    118.0   8.0    72      5    2
2   12.0    149.0  12.6    74      5    3
3   18.0    313.0  11.5    62      5    4
4   18.0    313.0  14.3    56      5    5

Heart Disease Dataset:
Shape: (303, 14)
Missing values:
 age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

First 5 rows:
    age  sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  \
0   63    1   3     145   233    1        0       150     0      2.3    0   
1   37    1   2     130   250    0        1       187     0      3.5    0   
2   41    0   1     130   204    0        0    

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7176\2523387564.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  air.fillna(method='pad', inplace=True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_7176\2523387564.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  heart.fillna(method='pad', inplace=True)


In [27]:
# === 3. Data Integration ===
# Create a synthetic common key for integration
air['ID'] = range(1, len(air) + 1)
heart['ID'] = range(1, len(heart) + 1)

merged = pd.merge(air, heart, on='ID')
print("Merged Dataset Shape:", merged.shape)

# Save to HTML
merged.to_html("merged_dataset.html", index=False)
print("Merged data saved to 'merged_dataset.html'")

Merged Dataset Shape: (153, 21)
Merged data saved to 'merged_dataset.html'


In [31]:
# === 4. Data Transformation ===

columns_to_transform = ['Ozone', 'Solar.R', 'Wind', 'Temp', 'Month', 'Day']
columns_to_transform = [col for col in columns_to_transform if col in air.columns]

# Standard Scaling
scaler_std = StandardScaler()
standard_scaled = scaler_std.fit_transform(air[columns_to_transform])
standard_df = pd.DataFrame(standard_scaled, columns=columns_to_transform)
print("\nStandard Scaled Data (first 5 rows):")
print(standard_df.head())

# MinMax Scaling
scaler_minmax = MinMaxScaler()
minmax_scaled = scaler_minmax.fit_transform(air[columns_to_transform])
minmax_df = pd.DataFrame(minmax_scaled, columns=columns_to_transform)
print("\nMin-Max Scaled Data (first 5 rows):")
print(minmax_df.head())

# Binarization
binarizer = Binarizer(threshold=0.5)
binarized_data = binarizer.fit_transform(minmax_scaled)
binarized_df = pd.DataFrame(binarized_data, columns=columns_to_transform)
print("\nBinarized Data (first 5 rows):")
print(binarized_df.head())


Standard Scaled Data (first 5 rows):
      Ozone   Solar.R      Wind      Temp     Month       Day
0  0.038647  0.043885 -0.728332 -1.153490 -1.411916 -1.675504
1 -0.120304 -0.752559 -0.557464 -0.623508 -1.411916 -1.562324
2 -0.883271 -0.409645  0.752529 -0.411515 -1.411916 -1.449144
3 -0.692530  1.404477  0.439270 -1.683472 -1.411916 -1.335965
4 -0.692530  1.404477  1.236657 -2.319450 -1.411916 -1.222785

Min-Max Scaled Data (first 5 rows):
      Ozone   Solar.R      Wind      Temp  Month       Day
0  0.239521  0.559633  0.300000  0.268293    0.0  0.000000
1  0.209581  0.339450  0.331579  0.390244    0.0  0.033333
2  0.065868  0.434251  0.573684  0.439024    0.0  0.066667
3  0.101796  0.935780  0.515789  0.146341    0.0  0.100000
4  0.101796  0.935780  0.663158  0.000000    0.0  0.133333

Binarized Data (first 5 rows):
   Ozone  Solar.R  Wind  Temp  Month  Day
0    0.0      1.0   0.0   0.0    0.0  0.0
1    0.0      0.0   0.0   0.0    0.0  0.0
2    0.0      0.0   1.0   0.0    0.0  0.0

In [35]:
# === 5. Error Correction ===
# Replace outliers in 'Ozone' column (values beyond 3 standard deviations from mean)

ozone_mean = air['Ozone'].mean()
ozone_std = air['Ozone'].std()
threshold = 3 * ozone_std

# Count before correction
outliers_before = ((abs(air['Ozone'] - ozone_mean)) > threshold).sum()
print(f"\nNumber of Ozone outliers before correction: {outliers_before}")

# Apply correction
air['Ozone'] = np.where(abs(air['Ozone'] - ozone_mean) > threshold, ozone_mean, air['Ozone'])

# Count after correction (should be 0)
outliers_after = ((abs(air['Ozone'] - ozone_mean)) > threshold).sum()
print(f"Number of Ozone outliers after correction: {outliers_after}")

# Show sample values
print("\nSample 'Ozone' values after correction:")
print(air['Ozone'].head())


Number of Ozone outliers before correction: 0
Number of Ozone outliers after correction: 0

Sample 'Ozone' values after correction:
0    41.0
1    36.0
2    12.0
3    18.0
4    18.0
Name: Ozone, dtype: float64


In [37]:
# === 6. Model Building (Linear Regression on Air Quality) ===

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Features and target
X = air[['Ozone']]
Y = air['Temp']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model initialization and training
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation metrics
print("R-squared Score:", r2_score(y_test, y_pred))
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

# Sorted X for a smooth regression line
X_sorted = X.sort_values(by='Ozone')
plt.scatter(X, Y, color='skyblue', label='Actual Data')
plt.plot(X_sorted, model.predict(X_sorted), color='red', label='Regression Line')
plt.xlabel('Ozone')
plt.ylabel('Temperature')
plt.title('Linear Regression: Ozone vs Temperature')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

R-squared Score: 0.359854956715416
Mean Absolute Error: 4.562534302008128
Mean Squared Error: 45.458957683572464
